# Caching Data

Spark offers the possibility to cache data, which means that it tries to keep (intermediate) results either in memory or on disk. This can be very helpful in iterative algorithms or interactive analysis, where you want to prevent that the same processing steps are performed over and over again.

### Approach to Caching
Instead of performing timings of individual executions, we use the `explain()` method again to see how output changes with cached intermediate results.

### Weather Example
We will again use the weather example to understand how caching works.

# 1. Load Data

First we load the weather data, which consists of the measurement data and some station metadata.

In [1]:
storageLocation = "s3://dimajix-training/data/weather"

## 1.1 Load Measurements

Measurements are stored in multiple directories (one per year)

In [2]:
from pyspark.sql.functions import *

# Union all years together
raw_weather = spark.read.text(storageLocation + "/2003").withColumn("year", lit(2003))    

### Extract Measurements

Measurements were stored in a proprietary text based format, with some values at fixed positions. We need to extract these values with a simple `SELECT` statement.

In [3]:
weather = raw_weather.select(
    col("year"),
    substring(col("value"),5,6).alias("usaf"),
    substring(col("value"),11,5).alias("wban"),
    substring(col("value"),16,8).alias("date"),
    substring(col("value"),24,4).alias("time"),
    substring(col("value"),42,5).alias("report_type"),
    substring(col("value"),61,3).alias("wind_direction"),
    substring(col("value"),64,1).alias("wind_direction_qual"),
    substring(col("value"),65,1).alias("wind_observation"),
    (substring(col("value"),66,4).cast("float") / lit(10.0)).alias("wind_speed"),
    substring(col("value"),70,1).alias("wind_speed_qual"),
    (substring(col("value"),88,5).cast("float") / lit(10.0)).alias("air_temperature"),
    substring(col("value"),93,1).alias("air_temperature_qual")
)
    
weather.limit(10).toPandas()

,year,usaf,wban,date,time,report_type,wind_direction,wind_direction_qual,wind_observation,wind_speed,wind_speed_qual,air_temperature,air_temperature_qual
0,2003,703160,25624,20030101,0000,SY-MT,010,5,N,5.2,5,-0.6,5
1,2003,703160,25624,20030101,0017,FM-16,020,1,N,4.6,1,-2.0,1
2,2003,703160,25624,20030101,0053,FM-15,010,5,N,5.2,5,-2.8,5
3,2003,703160,25624,20030101,0100,NSRDB,999,9,9,999.9,9,999.9,9
4,2003,703160,25624,20030101,0153,FM-15,010,5,N,6.2,5,-2.2,5
5,2003,703160,25624,20030101,0200,NSRDB,999,9,9,999.9,9,999.9,9
6,2003,703160,25624,20030101,0253,FM-15,010,5,N,7.2,5,-3.3,5
7,2003,703160,25624,20030101,0300,NSRDB,999,9,9,999.9,9,999.9,9
8,2003,703160,25624,20030101,0353,FM-15,020,5,N,6.2,5,-1.1,5
9,2003,703160,25624,20030101,0400,NSRDB,999,9,9,999.9,9,999.9,9


## 1.2 Load Station Metadata

We also need to load the weather station meta data containing information about the geo location, country etc of individual weather stations.

In [4]:
stations = spark.read \
    .option("header", True) \
    .csv(storageLocation + "/isd-history")

# Display first 10 records    
stations.limit(10).toPandas()

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,007005,99999,CWOS 07005,None,None,None,None,None,None,20120127,20120127
1,007011,99999,CWOS 07011,None,None,None,None,None,None,20111025,20121129
2,007018,99999,WXPOD 7018,None,None,None,+00.000,+000.000,+7018.0,20110309,20130730
3,007025,99999,CWOS 07025,None,None,None,None,None,None,20120127,20120127
4,007026,99999,WXPOD 7026,AF,None,None,+00.000,+000.000,+7026.0,20120713,20141120
5,007034,99999,CWOS 07034,None,None,None,None,None,None,20121024,20121106
6,007037,99999,CWOS 07037,None,None,None,None,None,None,20111202,20121125
7,007044,99999,CWOS 07044,None,None,None,None,None,None,20120127,20120127
8,007047,99999,CWOS 07047,None,None,None,None,None,None,20120613,20120717
9,007052,99999,CWOS 07052,None,None,None,None,None,None,20121129,20121130


# 2 Caching Data

For analysing the impact of cachign data, we will use a slightly simplified variant of the weather analysis (only temperature will be aggregated). We will change the execution by caching intermediate results and watch how execution plans change.

## 2.1 Original Execution Plan

First let's have the execution plans of the original query as our reference.

In [6]:
joined_weather = weather.join(stations, (weather.usaf == stations.USAF) & (weather.wban == stations.WBAN))
result = joined_weather.groupBy(joined_weather.CTRY, joined_weather.year).agg(
        min(when(joined_weather.air_temperature_qual == lit(1), joined_weather.air_temperature)).alias('min_temp'),
        max(when(joined_weather.air_temperature_qual == lit(1), joined_weather.air_temperature)).alias('max_temp')
    )

result.explain(True)

== Parsed Logical Plan ==
'Aggregate [CTRY#45, year#2], [CTRY#45, year#2, min(CASE WHEN (air_temperature_qual#16 = 1) THEN air_temperature#15 END) AS min_temp#245, max(CASE WHEN (air_temperature_qual#16 = 1) THEN air_temperature#15 END) AS max_temp#247]
+- AnalysisBarrier
      +- Join Inner, ((usaf#5 = USAF#42) && (wban#6 = WBAN#43))
         :- Project [year#2, substring(value#0, 5, 6) AS usaf#5, substring(value#0, 11, 5) AS wban#6, substring(value#0, 16, 8) AS date#7, substring(value#0, 24, 4) AS time#8, substring(value#0, 42, 5) AS report_type#9, substring(value#0, 61, 3) AS wind_direction#10, substring(value#0, 64, 1) AS wind_direction_qual#11, substring(value#0, 65, 1) AS wind_observation#12, (cast(cast(substring(value#0, 66, 4) as float) as double) / cast(10.0 as double)) AS wind_speed#13, substring(value#0, 70, 1) AS wind_speed_qual#14, (cast(cast(substring(value#0, 88, 5) as float) as double) / cast(10.0 as double)) AS air_temperature#15, substring(value#0, 93, 1) AS air_tempe

## 2.2 Caching Weather

First let us simply cache the joined input DataFrame.

In [7]:
joined_weather.cache()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string, USAF: string, WBAN: string, STATION NAME: string, CTRY: string, STATE: string, ICAO: string, LAT: string, LON: string, ELEV(M): string, BEGIN: string, END: string]

### Forcing physical caching

The `cache()` method again works lazily and only marks the DataFrame to be cached. The physical cache itself will only take place once the elements are evaluated. A common and easy way to enforce this is to call a `count()` on the to-be cached DataFrame.

In [8]:
joined_weather.count()

1798753

### Execution Plan with Cache

Now let us have a look at the execution plan with the cache for the `weather` DataFrame enabled.

In [9]:
result = joined_weather.groupBy(joined_weather.CTRY, joined_weather.year).agg(
        min(when(joined_weather.air_temperature_qual == lit(1), joined_weather.air_temperature)).alias('min_temp'),
        max(when(joined_weather.air_temperature_qual == lit(1), joined_weather.air_temperature)).alias('max_temp')
    )

result.explain(True)

== Parsed Logical Plan ==
'Aggregate [CTRY#45, year#2], [CTRY#45, year#2, min(CASE WHEN (air_temperature_qual#16 = 1) THEN air_temperature#15 END) AS min_temp#563, max(CASE WHEN (air_temperature_qual#16 = 1) THEN air_temperature#15 END) AS max_temp#565]
+- AnalysisBarrier
      +- Join Inner, ((usaf#5 = USAF#42) && (wban#6 = WBAN#43))
         :- Project [year#2, substring(value#0, 5, 6) AS usaf#5, substring(value#0, 11, 5) AS wban#6, substring(value#0, 16, 8) AS date#7, substring(value#0, 24, 4) AS time#8, substring(value#0, 42, 5) AS report_type#9, substring(value#0, 61, 3) AS wind_direction#10, substring(value#0, 64, 1) AS wind_direction_qual#11, substring(value#0, 65, 1) AS wind_observation#12, (cast(cast(substring(value#0, 66, 4) as float) as double) / cast(10.0 as double)) AS wind_speed#13, substring(value#0, 70, 1) AS wind_speed_qual#14, (cast(cast(substring(value#0, 88, 5) as float) as double) / cast(10.0 as double)) AS air_temperature#15, substring(value#0, 93, 1) AS air_tempe

In [10]:
result.toPandas()

,CTRY,year,min_temp,max_temp
0,DA,2003,-16.0,30.0
1,EZ,2003,-16.0,37.0
2,JA,2003,-0.7,34.2
3,NL,2003,-14.3,36.0
4,LU,2003,-13.0,37.4
5,SW,2003,-40.2,28.6
6,AU,2003,-17.6,37.4
7,SC,2003,21.0,33.0
8,UK,2003,-7.2,37.4
9,BE,2003,-11.3,36.3


### Remarks

Although the data is already cached, the execution plan still contains all steps. But the caching step won't be executed any more (since data is already cached), it is only mentioned here for completenss of the plan. We will see in the web interface.

The cache itself is presented as two steps in the execution plan:
* Creating the cache (InMemoryRelation)
* Using the cache (InMemoryTableScan)

If you look closely at the execution plans and compare these to the original uncached plan, you will notice that certain optimizations are not performed any more:
* Cache contains ALL columns of the weather DataFrame, although only a subset is required.
* Filter operation of JOIN is performed part of caching.

Caching is an optimization barrier. This means that Spark can only optimize plans before building the cache and plans after using the cache. No optimization is possible that spans building and using the cache. The idea simply is that the DataFrame should be cached exactly how it was specified without any column truncating or record filtering in place which appears after the cache.

## 2.2 Uncaching Data

Caches occupy resources (memory and/or disk). Once you do not need the cache any more, you'd probably like to free up the resources again. This is easily possible with the `unpersist()` method.

In [11]:
joined_weather.unpersist()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string, USAF: string, WBAN: string, STATION NAME: string, CTRY: string, STATE: string, ICAO: string, LAT: string, LON: string, ELEV(M): string, BEGIN: string, END: string]

### Exeuction plan after unpersist

Now we'd expect to have the original execution plan again. But for some reason (bug?) we don't get that any more:

In [14]:
result = joined_weather.groupBy(joined_weather.CTRY, joined_weather.year).agg(
        min(when(joined_weather.air_temperature_qual == lit(1), joined_weather.air_temperature)).alias('min_temp'),
        max(when(joined_weather.air_temperature_qual == lit(1), joined_weather.air_temperature)).alias('max_temp')
    )

result.explain(False)

== Physical Plan ==
*(3) HashAggregate(keys=[CTRY#45, 2003#726], functions=[min(CASE WHEN (cast(air_temperature_qual#16 as int) = 1) THEN air_temperature#15 END), max(CASE WHEN (cast(air_temperature_qual#16 as int) = 1) THEN air_temperature#15 END)])
+- Exchange hashpartitioning(CTRY#45, 2003#726, 200)
   +- *(2) HashAggregate(keys=[CTRY#45, 2003 AS 2003#726], functions=[partial_min(CASE WHEN (cast(air_temperature_qual#16 as int) = 1) THEN air_temperature#15 END), partial_max(CASE WHEN (cast(air_temperature_qual#16 as int) = 1) THEN air_temperature#15 END)])
      +- *(2) Project [air_temperature#15, air_temperature_qual#16, CTRY#45]
         +- *(2) BroadcastHashJoin [usaf#5, wban#6], [USAF#42, WBAN#43], Inner, BuildRight
            :- *(2) Project [substring(value#0, 5, 6) AS usaf#5, substring(value#0, 11, 5) AS wban#6, (cast(cast(substring(value#0, 88, 5) as float) as double) / 10.0) AS air_temperature#15, substring(value#0, 93, 1) AS air_temperature_qual#16]
            :  +- *(2)

### Remarks

As you see in the execution plan, the cache has been removed now and the plan equals to the original one before we started caching data.

# 3 Cache Levels

Spark supports different levels of cache (memory, disk and a combination). These can be specified explicitly if you use `persist()` instead of `cache()`. Cache actually is a shortcut for `persist(MEMORY_AND_DISK)`.

In [ ]:
from pyspark.storagelevel import StorageLevel

weather.persist(StorageLevel.MEMORY_ONLY)
weather.persist(StorageLevel.MEMORY_ONLY_SER)
weather.persist(StorageLevel.DISK_ONLY)
weather.persist(StorageLevel.MEMORY_AND_DISK)

weather.persist(StorageLevel.MEMORY_ONLY_2)
weather.persist(StorageLevel.MEMORY_ONLY_SER_2)
weather.persist(StorageLevel.DISK_ONLY_2)
weather.persist(StorageLevel.MEMORY_AND_DISK_2)


### Cache level explanation

* `MEMORY_ONLY` - stores all records directly in memory
* `MEMORY_ONLY_SER` - stores all records serialized in memory. Should use less memory, but requires additional work by the CPU
* `DISK_ONLY` - stores all records serialized on disk
* `MEMORY_AND_DISK` - stores all records first in memory and spills onto disk when no space is left in memory
* `..._2` - stores caches on two nodes instead of one for additional redundancy